In [1]:
import numpy as np
import pandas as pd

import datetime
import time
import hashlib
import json
import sys

import random


In [2]:
sys.path.append('/Users/eyal42/Work/Entrepreneurship/Ramzgate/Hackathon/NodeNeighborhood/priority')


In [3]:
import RandomNetwork

In [4]:
def getID(tt,task):
    string=tt.__str__()+task['op']+task['node']
    m = hashlib.sha1()
    m.update(string.encode('utf-8'))
    return(m.hexdigest())

def createTask(op,node):
    return({'op':op,'node':node})

def createJob(task):
    tt=datetime.datetime.now()
    return({'id':getID(tt,task),'creation_time':tt,'task':tsk})

In [5]:
#################################################################################################################
# Delegator node events:
#        1. creation -  when a node is added to the priority list (or readded after being deleted)
#        2. connection -  when a worker is assigned a 'CNCT' job to the node and the job is completed
#        3. pong - when a node appears in the list of neighbors returned from a 'CNCT' job for a different node
#################################################################################################################

class Node:

    def __init__(self,nid):
        tt=datetime.datetime.now()
        ut=datetime.datetime.timestamp(tt)
        self.id=nid
        self.doc={'id':nid,'loc':None,'neigh':[]}
        self.creation=ut
        self.last_connect=None
        self.last_pong=ut
        self.priority=ut
        self.log={'CNCT':0,'PONG':0}
        return
    
    def loadDoc(self,nodeDoc):
        assert self.id==nodeDoc['id'],"Id mismatch"
        self.doc=nodeDoc
        return
    
    def connect(self):
        tt=datetime.datetime.now()
        ut=datetime.datetime.timestamp(tt)
        self.last_connect=ut
        self.priority=ut
        self.log['CNCT']+=1
        return
        
    def pong(self):
        tt=datetime.datetime.now()
        ut=datetime.datetime.timestamp(tt)
        self.last_pong=ut
        self.priority=0.7*ut+0.3*self.priority  # (ut-self.priority)*0.7+self.priority
        self.log['PONG']+=1
        return
    
    #def __str__(self):
    #    return(json.dumps(self.doc)+'--'+self.priority.__str__()+'--'+self.last_pong.__str__()+'--'+self.last_connect.__str__())
    
    def __str__(self):
        jj={
            'node_id':self.id,
            'priority':self.priority.__str__(),
            'last_pong':self.last_pong.__str__(),
            'last_connect':self.last_connect.__str__(),
            'doc':self.doc
        }
        return(json.dumps(jj,indent=2))

    
    def __json__(self):
        return(self.doc)
    


In [6]:
class NodeList:
    
    def __init__(self):
        self.nodeList={}
        return

    def inNodeList(self,nid):
        return(nid in self.nodeList)

    def insert(self,node):
        self.nodeList[node.id]=node
        return
    
    def remove(self,nid):
        del(self.nodeList[nid])
        
    def updateNode(self,nid,update):
        if update=='CNCT':
            self.nodeList[nid].connect()
        elif update=='PONG':
            self.nodeList[nid].pong()
        else:
            raise Exception("Unknown update")
        return
        
    def getHead(self):
        tmp={self.nodeList[nid].priority:self.nodeList[nid].id for nid in self.nodeList}
        return(tmp[min(tmp)])

    def getTail(self):
        tmp={self.nodeList[nid].priority:self.nodeList[nid].id for nid in self.nodeList}
        return(tmp[max(tmp)])

    def getNode(self,nid):
        return(self.nodeList[nid])

    def showPriority(self):
        print('length=',len(self.nodeList))
        tmp={self.nodeList[nid].priority:nid for nid in self.nodeList}
        tmp=dict(sorted(tmp.items()))
        for pp in tmp:
            nid=tmp[pp]
            print(pp,nid,self.nodeList[nid].log)
        return


In [8]:
nt=RandomNetwork.Network(100,0.001)
nl=NodeList()
bootstrap='0x00000000000000000000'
#bdoc=nt.query(bootstrap)
nl.insert(Node(bootstrap))
nl.showPriority()

length= 1
1702996761.55318 0x00000000000000000000 {'CNCT': 0, 'PONG': 0}


In [210]:
hid=nl.getHead()

for nid in nl.getNode(hid).doc['neigh']:
    if nl.inNodeList(nid):
        nl.updateNode(nid,'PONG')
    else:
        ndoc=nt.query(nid)
        nl.insert(Node(ndoc))
nl.updateNode(hid,'CNCT')
#nl.showPriority()

In [211]:
print(nl.nodeList['0x00000000000000000000'])

{
  "node_id": "0x00000000000000000000",
  "priority": "1702966056.87965",
  "last_pong": "1702966055.687204",
  "last_connect": "1702966056.87965",
  "doc": {
    "id": "0x00000000000000000000",
    "loc": null,
    "neigh": []
  }
}


In [7]:
class Job:
    
    def __init__(self,node,task):
        self.created_time=datetime.datetime.now()
        self.start_time=None
        self.end_time=None
        self.node=node
        self.task=task
        self.jid=self.__getID__()
        self.status='created'
        
    def __getID__(self):
        string=self.created_time.__str__()+self.node.__str__()+self.task
        m = hashlib.sha256()
        m.update(string.encode('utf-8'))
        return(m.hexdigest())
    
    def __str__(self):
        jj={
            'job_id':self.jid,
            'created_time':self.created_time.__str__(),
            'start_time':self.start_time.__str__(),
            'end_time':self.end_time.__str__(),
            'task':self.task,
            'node':self.node.__json__(),
            'status':self.status
        }
        return(json.dumps(jj,indent=2))


In [8]:
class Controller:
    
    def __init__(self):
        self.nl=NodeList()
        return


    def initialize(self):
        bootstrap='0x00000000000000000000'
        self.nl.insert(Node(bootstrap))
        return
        
    def delegate(self):
        hid=self.nl.getHead()
        node=self.nl.getNode(hid)        
        self.nl.remove(hid) # possibly change this to a 'pull' or 'pop'
        jb=Job(node,'CNCT')
        return(jb)
        
    def ingest(self,jb):
        self.nl.insert(jb.node)
        for nid in jb.node.doc['neigh']:
            if self.nl.inNodeList(nid):
                self.nl.updateNode(nid,'PONG')
            else:
                self.nl.insert(Node(nid))
        self.nl.updateNode(jb.node.id,'CNCT')
        return


In [14]:
class Worker:

    def __init__(self,nt):
        self.nt=nt
        return

    def executeJob(self,jb):
        jb.start_time=datetime.datetime.now()
        jb.status='started'
        nid=jb.node.id
        ndoc=self.nt.query(nid)
        jb.node.doc=ndoc
        jb.end_time=datetime.datetime.now()
        jb.status='completed'
        return(jb)


In [12]:
## Worker Test
nt=RandomNetwork.Network(100,1)
wrkr=Worker(nt)
nn=Node('0x00000000000000000000')
#nn=Node('0xed86f246f734646d674d')
#print(nn)
jb0=Job(nn,'CNCT')
#print(jb0)
jb1=wrkr.executeJob(jb0)
#print(jb1)

In [15]:
class NtSim:
    def __init__(self,size):
        self.size=size
        self.nt=RandomNetwork.Network(self.size,query_time_scale=0.01)
        self.cntrl=Controller()
        self.wrkr=Worker(self.nt)
        return
    
    def Start(self):
        self.cntrl.initialize()
        return
    
    def Iterate(self):
        jb0=self.cntrl.delegate()
        #print(jb0)
        jb1=self.wrkr.executeJob(jb0)
        #print(jb1)
        self.cntrl.ingest(jb1)
        return


In [29]:
ntsim=NtSim(10000)
ntsim.Start()
for ii in range(500):
    ntsim.Iterate()
    #print(len(ntsim.cntrl.nl.nodeList))
ntsim.cntrl.nl.showPriority()


length= 5167
1703003138.2264667 0xb5f5f6353725137833fc {'CNCT': 0, 'PONG': 1}
1703003138.229423 0xd566dce13cebefa2c17e {'CNCT': 0, 'PONG': 0}
1703003138.229469 0xd10269834720306860cc {'CNCT': 0, 'PONG': 0}
1703003138.229496 0x1b14eff471578da3525c {'CNCT': 0, 'PONG': 0}
1703003138.235754 0xaeaa8a872d9de96a7c8f {'CNCT': 0, 'PONG': 0}
1703003138.235798 0x3fa79e5d234b35dc7ebd {'CNCT': 1, 'PONG': 0}
1703003138.237097 0x720280228d832ca4d96e {'CNCT': 0, 'PONG': 1}
1703003138.2413995 0x55508920d2dacc86c03d {'CNCT': 0, 'PONG': 2}
1703003138.241455 0xd45479e1c9d5a14f59f4 {'CNCT': 0, 'PONG': 0}
1703003138.241488 0x3136c650c779f85580e6 {'CNCT': 0, 'PONG': 0}
1703003138.241581 0x8316dd6a664c6cc1ece8 {'CNCT': 0, 'PONG': 0}
1703003138.241614 0x364f5a1d50a658ff06ad {'CNCT': 0, 'PONG': 0}
1703003138.241627 0xd3a40c174837951984b7 {'CNCT': 0, 'PONG': 0}
1703003138.241668 0xb93ff43146bedca7ceea {'CNCT': 0, 'PONG': 0}
1703003138.241737 0x48eadcd51ef76fea028f {'CNCT': 0, 'PONG': 0}
1703003138.2419968 0x9ee0

In [19]:
len(ntsim.nt.nt)

6331

In [30]:
print(ntsim.cntrl.nl.getNode('0x1534f3d84cd131163064'))

{
  "node_id": "0x1534f3d84cd131163064",
  "priority": "1703003142.5702329",
  "last_pong": "1703003142.575952",
  "last_connect": "1703003138.169988",
  "doc": {
    "id": "0x1534f3d84cd131163064",
    "loc": "Gardez",
    "neigh": [
      "0x973b022a25d675f36557",
      "0x3eac69f45044ab34a31c",
      "0xfaa414d679fc9185effc",
      "0x236d00876b4372632c60",
      "0x5b88bcc2c54c2e94c2e9",
      "0x81110df80ca4086e306c",
      "0x502845c58c2c863cc563",
      "0x88a1bd763690de16bab9",
      "0x1c38fb075750a6e357e2",
      "0xdb542d20d7dff701d146",
      "0xcb0a72bbffcd78aaf1c6",
      "0xb749475c2b2f437d28cf",
      "0x2145c3e7ce74ef291f72",
      "0xd5f12e53a182c062b6bf",
      "0xce1ef87677d0f2daddcd",
      "0x5ef66e234a8aeb113b16",
      "0x9a1aef7972eb212b505f",
      "0xbaa924ce1ee3617f30a8",
      "0x671cecdd43a2b519e266",
      "0xa99ff50a8d5bd117882f",
      "0x1fca884ab7f76f68ae1b",
      "0x6d7084b4994a00d15257",
      "0x704abeb8865b3e90b165",
      "0x89d72f557a3a9a0cf250

In [35]:
nnt=RandomNetwork.Network(20000,1)
len(nnt.nt)

8681